## 1. 라이브러리 로드

In [12]:
import os
import cv2
import json
import imgaug as ia
import imgaug.augmenters as iaa

from tqdm import tqdm # progress bar 시각화 라이브러리
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

## 2. 경로 설정

In [51]:
input_image_path = "C:\\Users\\rhoms\\businside\\bus_passenger\\Validation\\img_allInOne"

input_json_path = "data/val.json"
output_image_path = "data/aug_val_img"
output_json_path = "data/json"
output_json_path = os.path.join(output_json_path, "aug_val.json")

# JSON 파일에서 COCO 데이터 불러오기
with open(input_json_path, "r") as f:
    coco_data = json.load(f)

## 3. 이미지, json 파일 잘 로드되었는지 확인하기

In [52]:
# 이미지 정보 가져오기
image_info = coco_data["images"][0]

# 이미지 파일 경로 생성
image_path = os.path.join(input_image_path, image_info["file_name"])
print()
# 이미지 파일 읽어오기
# Image Shape: (1080, 1920, 3) 이렇게 출력나와야 함
if os.path.exists(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        print("Image Shape:", image.shape)
    else:
        print("Failed to read image:", image_path)
else:
    print("Image file not found:", image_path)

# input_image_path 경로 내 첫 번째 이미지 파일명 가져오기
image_files = os.listdir(input_image_path)
if image_files:
    first_image_filename = image_files[0]
    print("첫 번째 이미지 파일명:", first_image_filename)
else:
    print("이미지 파일이 없습니다.")


Image file not found: C:\Users\rhoms\businside\bus_passenger\Validation\img_allInOne\[district]leave_634C_96.jpg
첫 번째 이미지 파일명: [emergency]accident11_087A_0.jpg


In [53]:
#print("coco data: ", coco_data)

In [54]:
image_info = coco_data["images"][0]

image_path = os.path.join(input_image_path, image_info["file_name"])
image = cv2.imread(image_path)
print(image_info)


{'height': 1080, 'width': 1920, 'id': 1, 'file_name': '[district]leave_634C_96.jpg'}


In [55]:
print("Image Path:", image_path)
image = cv2.imread(image_path)
print("Image:", image)
# 이미지 3차원으로 나와야 함

Image Path: C:\Users\rhoms\businside\bus_passenger\Validation\img_allInOne\[district]leave_634C_96.jpg
Image: None


In [56]:
# print("Image Width:", image.shape[1])
# print("Image Height:", image.shape[0])

## 4. Augmentation

In [57]:
# 증강 기법 설정
seq = iaa.Sequential([
    iaa.Fliplr(1.0),  # 좌우 반전
    iaa.GaussianBlur(sigma=(0, 1.0)),  # 가우시안 블러
    iaa.MultiplyBrightness((0.7, 1.3)),  # 밝기 조절
    iaa.MultiplySaturation((0.5, 1.5)),  # 채도 조절
    iaa.WithChannels(0, iaa.Add((-30, 30))),  # 색상 조정
    iaa.ChangeColorspace(from_colorspace="HSV", to_colorspace="RGB"),
    iaa.AdditiveGaussianNoise(scale=(0, 0.1 * 255))  # 가우시안 노이즈 추가
])

# 모든 증강 기법 순차적으로 적용
for idx, image_info in tqdm(enumerate(coco_data["images"]), desc="Processing images", total=len(coco_data["images"])):
    image_filename = os.path.basename(image_info["file_name"])
    image_path = os.path.join(input_image_path, image_filename)
    image = cv2.imread(image_path)

    if image is None:
        continue

    annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] == image_info["id"]]

    if not annotations:
        continue

    for i in range(len(seq)):
        augmented_image = seq[i](image=image)
        
        for ann in annotations:
            bbox = BoundingBox(x1=ann["bbox"][0], y1=ann["bbox"][1], x2=ann["bbox"][0] + ann["bbox"][2], y2=ann["bbox"][1] + ann["bbox"][3])
            augmented_annotations = [bbox]

            new_image_info = image_info.copy()
            new_image_info["file_name"] = f"{image_filename.split('.')[0]}_{i+1}.jpg"
            new_image_info["id"] = len(coco_data["images"]) + 1

            new_ann = ann.copy()
            new_ann["bbox"] = [
                augmented_annotations[0].x1,  # x1 좌표
                augmented_annotations[0].y1,  # y1 좌표
                augmented_annotations[0].x2 - augmented_annotations[0].x1,  # 너비
                augmented_annotations[0].y2 - augmented_annotations[0].y1  # 높이
            ]

            # 중복되지 않은 annotation id 생성
            new_annotation_id = ann["id"] + len(coco_data["annotations"])
            while new_annotation_id in [ann["id"] for ann in coco_data["annotations"]]:
                new_annotation_id += 1
            new_ann["id"] = new_annotation_id
            new_ann["image_id"] = new_image_info["id"]

            coco_data["images"].append(new_image_info)
            coco_data["annotations"].append(new_ann)

            # 이미지 저장
            output_image_file = os.path.join(output_image_path, new_image_info["file_name"])
            cv2.imwrite(output_image_file, augmented_image)

# 수정된 COCO 데이터 형식을 JSON 파일로 저장
with open(output_json_path, "w") as f:
    json.dump(coco_data, f, indent=4)

print("finish!")


Processing images: 20561it [20:22, 16.82it/s]                                                                          


finish!
